In [ ]:
# Set this to wherever your organelle reaction data is stored
home = %env HOME
organelle_rxn_data = "{}/organelle/organelle_files/mcell/output_data/react_data/".format(home)
organelle_rxn_data

In [ ]:
import glob
import pandas as pd
import cufflinks as cf
import plotly
import plotly.offline as py
import os

# Set offline to True or you'll have to create a plotly account
cf.set_config_file(offline=True, world_readable=False, theme='pearl')
py.init_notebook_mode() # graphs charts inline 

In [ ]:
# Create a list of dataframes, one for every reaction data output file
# e.g., ./seed_0001/a.World.dat

rxndata_fnames = list(glob.iglob("{}/**/*.World.dat".format(organelle_rxn_data)))
rxndata_fnames.sort()

df_list = []
for fname in rxndata_fnames:
    seed = int(fname.split('/')[-2][5:])
    basename = os.path.basename(fname)
    spec_loc_name, _ = os.path.splitext(basename)
    spec_loc_name = spec_loc_name.replace('.','_')
    df = pd.read_csv(fname, names=['time', "s{}/{}".format(seed, spec_loc_name)], delim_whitespace=True)
    df = df.set_index('time')
    df_list.append(df)

In [ ]:
# Turn all the dataframes into a single dataframe
df_rxndata = pd.concat(df_list, axis=1)

In [ ]:
# To make loc indexing work better, we'll remove the time column
# We can save the time column to a series in case we want it later
df_rxndata = df_rxndata.reset_index()
series_time = df_rxndata.pop('time')

In [ ]:
# Setting our columns to be a MultiIndex. This let's us have hierarchical columns names.
# For this example, we'll set the 'seed' to be the top level column name and the 'species'
# be the second level column name
df_rxndata.columns = pd.MultiIndex.from_tuples(
    [tuple(c.split('/')) for c in df_rxndata.columns],
    names=['seed', 'species'])
df_rxndata.tail()

In [ ]:
# Just an example of slicing with loc
df_rxndata.loc[900:1000:10]['s5']['b_World']

In [ ]:
# Here are a couple of ways to grab all seeds for one species, one with loc and one with iloc
df_a = df_rxndata.loc[:,(slice(None), 'a_World')]
#df_a = df_rxndata.iloc[:, df_rxndata.columns.get_level_values(1)=='a_World']
df_a.tail()

In [ ]:
# Plot all our "a" species, seeds 1 through 10
df_a.iplot()

In [ ]:
# Take the mean of all the "a" seeds and plot them
a_avg = df_a.mean(axis=1)
a_avg.iplot()

In [ ]:
# Create and plot a dataframe containing the average of every species
avg_list = []
for species in df_rxndata.columns.levels[1]:
    avg_list.append(df_rxndata.loc[:,(slice(None), species)].mean(axis=1))
df_avg = pd.concat(avg_list, axis=1)
df_avg.iplot()